In [ ]:
def get_today_date():
    from datetime import datetime
    date_str = str(datetime.now())
    today=datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
    return today.strftime('%d-%b-%Y')

def get_option_chain(symbol):
    today = get_today_date()
    # today = '28-Sep-2023'
    # filename_OI_Chng = symbol + '-CHNG-IN-OI-' + today + ".csv"
    # filename_OI_Total =  symbol + '-TOTAL-OI-' + today + ".csv"
    # filename_ce_option_chain =  symbol +'-CE-OPTION-CHAIN-' + today + ".csv"
    # filename_pe_option_chain =  symbol +'-PE-OPTION-CHAIN-' + today + ".csv"
    # filename_ce_full_option_chain =  symbol +'-CE-FULL-OPTION-CHAIN-' + today + ".csv"
    # filename_pe_full_option_chain =  symbol +'-PE-FULL-OPTION-CHAIN-' + today + ".csv"
    
    outdir = "D:\Jupyter Python\Stocks-office\json-csv\\"
    
    file_OI_Chng = symbol + '-CHNG-IN-OI-' + today + ".csv"
    file_OI_Total =  symbol + '-TOTAL-OI-' + today + ".csv"
    file_ce_option_chain =  symbol +'-CE-OPTION-CHAIN-' + today + ".csv"
    file_pe_option_chain =  symbol +'-PE-OPTION-CHAIN-' + today + ".csv"
    file_ce_full_option_chain =  symbol +'-CE-FULL-OPTION-CHAIN-' + today + ".csv"
    file_pe_full_option_chain =  symbol +'-PE-FULL-OPTION-CHAIN-' + today + ".csv"
    
    filename_OI_Chng = os.path.join(outdir, file_OI_Chng)
    filename_OI_Total =  os.path.join(outdir, file_OI_Total)
    filename_ce_option_chain =  os.path.join(outdir, file_ce_option_chain)
    filename_pe_option_chain =  os.path.join(outdir, file_pe_option_chain)
    filename_ce_full_option_chain =  os.path.join(outdir, file_ce_full_option_chain)
    filename_pe_full_option_chain =  os.path.join(outdir, file_pe_full_option_chain)
   
    hd =pd.DataFrame({'Time':[],'CE_OI_CHNG':[],'PE_OI_CHNG':[]})
    hd.to_csv(filename_OI_Chng,header=True,index=False)
    
    hd2 =pd.DataFrame({'Time':[],'CE_OI_TOTAL':[],'PE_OI_TOTAL':[]})
    hd2.to_csv(filename_OI_Total,header=True,index=False)
   
    url='https://www.nseindia.com/api/option-chain-indices?symbol='+ symbol
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}
    starttime=time.time()
    timeout = time.time() + 60*420  # 60 seconds times 2 meaning the script will run for 2 minutes
    while time.time() <= timeout:
        try:
            r = requests.get(url,headers=headers)
            data=r.json()
           
            list_ce=[]
            list_pe=[]
            list_time=[]
            j=data["filtered"]["data"]
            dict_={'strikePrice': 0,                  
                 'expiryDate': '',
                 'underlying': 'BANKNIFTY',
                 'identifier': '',
                 'openInterest': 0,
                 'changeinOpenInterest': 0,
                 'pchangeinOpenInterest': 0,
                 'totalTradedVolume': 0,
                 'impliedVolatility': 0,
                 'lastPrice': 0,
                 'change': 0,
                 'pChange': 0,
                 'totalBuyQuantity': 0,
                 'totalSellQuantity': 0,
                 'bidQty': 0,
                 'bidprice': 0,
                 'askQty': 0,
                 'askPrice': 0,
                 'underlyingValue':0}

            for i in range(len(j)):
                list_time.append(data["records"]["timestamp"])
                try:
                    list_ce.append(data["filtered"]["data"][i]["CE"])
                except:
                    list_ce.append(dict_)
                try:
                    list_pe.append(data["filtered"]["data"][i]["PE"])
                except :
                    list_pe.append(dict_)
            df_ce=pd.DataFrame(list_ce)
            df_pe=pd.DataFrame(list_pe)
            df_time=pd.DataFrame(list_time,columns=["timestamp"])
            df_ce["timestamp"]=df_time["timestamp"]
            df_pe["timestamp"]=df_time["timestamp"]


            df_ce_reorder = df_ce[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                            'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
            df_ce_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                            'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
            df_pe_reorder = df_pe[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                            'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
            df_pe_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                            'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
            ce_oi_change=df_ce_reorder["oi_change"].sum()
            pe_oi_change=df_pe_reorder["oi_change"].sum()
            ce_oi_total=df_ce_reorder["oi"].sum()
            pe_oi_total=df_pe_reorder["oi"].sum()
            
            
            time1 = data["records"]["timestamp"]
            oi_change=pd.DataFrame({"Time":[time1],"CE_OI_CHNG":[ce_oi_change],"PE_OI_CHNG":[pe_oi_change]})
            oi_total=pd.DataFrame({"Time":[time1],"CE_OI_TOTAL":[ce_oi_total],"PE_OI_TOTAL":[pe_oi_total]})

            oi_change.to_csv(filename_OI_Chng,mode='a',header=False,index=False)
            oi_total.to_csv(filename_OI_Total,mode='a',header=False,index=False)

            df_ce_reorder.to_csv(filename_ce_option_chain,header=True,index=False)
            df_pe_reorder.to_csv(filename_pe_option_chain,header=True,index=False)
            
            df_ce_reorder.to_csv(filename_ce_full_option_chain,mode='a',header=False,index=False)
            df_pe_reorder.to_csv(filename_pe_full_option_chain,mode='a',header=False,index=False)

            time.sleep(180 - ((time.time() - starttime) % 180.0)) # 5 second interval between each new iteration
            # print(oi_change)
            # print(oi_total)
        except :
            print("Network error.....reconnecting...",starttime)
            
######################### Main program ######################

import pandas as pd
import requests
import os
import time
get_option_chain('BANKNIFTY')


Network error.....reconnecting... 1665460099.9545012
Network error.....reconnecting... 1665460099.9545012
Network error.....reconnecting... 1665460099.9545012
Network error.....reconnecting... 1665460099.9545012
Network error.....reconnecting... 1665460099.9545012
Network error.....reconnecting... 1665460099.9545012


In [ ]:
import pandas as pd
import requests
import time
symbol='BANKNIFTY'
url='https://www.nseindia.com/api/option-chain-indices?symbol='+ symbol
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}
    # starttime=time.time()
    # timeout = time.time() + 60*420  # 60 seconds times 2 meaning the script will run for 2 minutes
# while time.time() <= timeout:
    # try:
r = requests.get(url,headers=headers)
data=r.json()

list_ce=[]
list_pe=[]
list_time=[]
j=data["filtered"]["data"]
for i in range(len(j)):
    list_time.append(data["records"]["timestamp"])
    try:
        list_ce.append(data["filtered"]["data"][i]["CE"])
    except:
        list_ce.append(data["filtered"]["data"][i+1]["CE"])
    try:
        list_pe.append(data["filtered"]["data"][i]["PE"])
    except :
        list_pe.append(data["filtered"]["data"][i+1]["PE"])

df_ce=pd.DataFrame(list_ce)
df_pe=pd.DataFrame(list_pe)
df_time=pd.DataFrame(list_time,columns=["timestamp"])
df_ce["timestamp"]=df_time["timestamp"]
df_pe["timestamp"]=df_time["timestamp"]


df_ce_reorder = df_ce[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
df_ce_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
df_pe_reorder = df_pe[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
df_pe_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
ce_oi_change=df_ce_reorder["oi_change"].sum()
pe_oi_change=df_pe_reorder["oi_change"].sum()
ce_oi_total=df_ce_reorder["oi"].sum()
pe_oi_total=df_pe_reorder["oi"].sum()


time1 = data["records"]["timestamp"]
oi_change=pd.DataFrame({"Time":[time1],"CE_OI_CHNG":[ce_oi_change],"PE_OI_CHNG":[pe_oi_change]})
oi_total=pd.DataFrame({"Time":[time1],"CE_OI_TOTAL":[ce_oi_total],"PE_OI_TOTAL":[pe_oi_total]})


In [ ]:
len(df_ce)

In [ ]:
df_ce_reorder